# Mechanisms of Action (MoA): 作用機序

## Basic Study

### MoA

薬が治療効果を及ぼす仕組みのことです。通常、薬がどの標的分子（タンパク質）と相互作用し、どのような影響を与えた結果、治療効果が得られるのかを、分子レベルで言及します。ある薬に対して抵抗性になった場合、作用機序の異なる薬を使うことで、同じ治療効果が得られる可能性があります。

### 学習データ

- `g-x` は 遺伝子発現。772 列。遺伝子発現とは、遺伝子にコードされた情報がタンパク質分子の組み立てを指示するために利用されるプロセスである。細胞は、3つの塩基のグループで遺伝子の配列を読み取る。
- `c-x` は 細胞生存能。100 列。細胞生存率は、集団内の生きている健康な細胞の割合を測定するものです。細胞生存率アッセイは、細胞の全体的な健康状態を決定したり、培養や実験条件を最適化したり、薬物スクリーニングなどの化合物で処理した後の細胞生存率を測定したりするために使用されます。
- `cp_type` は 化合物（`cp_vehicle`）または対照摂動（`ctrl_vehicle`）で処理されたサンプルを示し、対照摂動は MoA を持たない
- `cp_time` と `cp_dose` は治療期間（24、48、72 時間）と投与量（高または低）

### 目的変数

- 206の作用機序への反応を予測する

In [2]:
import numpy as np
import pandas as pd

In [3]:
def df_stats(df):
    stats = []
    for col in df.columns:
        stats.append(
            (
                col,
                df[col].nunique(),
                df[col].value_counts().index[0],
                df[col].value_counts().values[0],
                df[col].isnull().sum() * 100 / df.shape[0],
                df[col].value_counts(normalize=True, dropna=False).values[0] * 100,
                df[col].dtype,
            )
        )
    return pd.DataFrame(
        stats, columns=["カラム名", "カラムごとのユニーク値数", "最も出現頻度の高い値", "最も出現頻度の高い値の出現回数", "欠損損値の割合", "最も多いカテゴリの割合", "Type"]
    )

In [4]:
train_df = pd.read_csv('../input/lish-moa/train_features.csv')
test_df = pd.read_csv('../input/lish-moa/test_features.csv')
target_df = pd.read_csv('../input/lish-moa/train_targets_scored.csv')
submit_df = pd.read_csv('../input/lish-moa/sample_submission.csv')
non_target_df = pd.read_csv('../input/lish-moa/train_targets_nonscored.csv')

In [5]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23814 entries, 0 to 23813
Columns: 876 entries, sig_id to c-99
dtypes: float64(872), int64(1), object(3)
memory usage: 159.2+ MB


In [6]:
train_df['cp_type'].value_counts()

trt_cp         21948
ctl_vehicle     1866
Name: cp_type, dtype: int64

In [7]:
train_df['cp_time'].value_counts()

48    8250
72    7792
24    7772
Name: cp_time, dtype: int64

In [8]:
train_df['cp_dose'].value_counts()

D1    12147
D2    11667
Name: cp_dose, dtype: int64

In [9]:
train_df.describe()

,cp_time,g-0,g-1,g-2,g-3,g-4,g-5,g-6,g-7,g-8,...,c-90,c-91,c-92,c-93,c-94,c-95,c-96,c-97,c-98,c-99
count,23814.000000,23814.000000,23814.000000,23814.000000,23814.000000,23814.000000,23814.000000,23814.000000,23814.000000,23814.000000,...,23814.000000,23814.000000,23814.000000,23814.000000,23814.000000,23814.000000,23814.000000,23814.000000,23814.000000,23814.000000
mean,48.020156,0.248366,-0.095684,0.152253,0.081971,0.057347,-0.138836,0.035961,-0.202651,-0.190083,...,-0.469244,-0.461411,-0.513256,-0.500142,-0.507093,-0.353726,-0.463485,-0.378241,-0.470252,-0.301505
std,19.402807,1.393399,0.812363,1.035731,0.950012,1.032091,1.179388,0.882395,1.125494,1.749885,...,2.000488,2.042475,2.001714,2.107105,2.159589,1.629291,2.059725,1.703615,1.834828,1.407918
min,24.000000,-5.513000,-5.737000,-9.104000,-5.998000,-6.369000,-10.000000,-10.000000,-10.000000,-10.000000,...,-10.000000,-10.000000,-10.000000,-10.000000,-10.000000,-10.000000,-10.000000,-10.000000,-10.000000,-10.000000
25%,24.000000,-0.473075,-0.562200,-0.437750,-0.429575,-0.470925,-0.602225,-0.493900,-0.525175,-0.511675,...,-0.566175,-0.565975,-0.589975,-0.568700,-0.563775,-0.567975,-0.552575,-0.561000,-0.592600,-0.562900
50%,48.000000,-0.008850,-0.046600,0.075200,0.008050,-0.026900,-0.015650,-0.000650,-0.017900,0.010000,...,-0.009900,0.003250,-0.009100,-0.013750,-0.003300,-0.010250,-0.001250,-0.006800,0.014000,-0.019500
75%,72.000000,0.525700,0.403075,0.663925,0.463400,0.465375,0.510425,0.528725,0.411900,0.549225,...,0.457750,0.461500,0.445675,0.452900,0.470900,0.444750,0.465225,0.446400,0.461275,0.438650
max,72.000000,10.000000,5.039000,8.257000,10.000000,10.000000,7.282000,7.333000,5.473000,8.887000,...,4.069000,3.960000,3.927000,3.596000,3.747000,2.814000,3.505000,2.924000,3.111000,3.805000


In [10]:
train_df.head()

,sig_id,cp_type,cp_time,cp_dose,g-0,g-1,g-2,g-3,g-4,g-5,...,c-90,c-91,c-92,c-93,c-94,c-95,c-96,c-97,c-98,c-99
0,id_000644bb2,trt_cp,24,D1,1.0620,0.5577,-0.2479,-0.6208,-0.1944,-1.0120,...,0.2862,0.2584,0.8076,0.5523,-0.1912,0.6584,-0.3981,0.2139,0.3801,0.4176
1,id_000779bfc,trt_cp,72,D1,0.0743,0.4087,0.2991,0.0604,1.0190,0.5207,...,-0.4265,0.7543,0.4708,0.0230,0.2957,0.4899,0.1522,0.1241,0.6077,0.7371
2,id_000a6266a,trt_cp,48,D1,0.6280,0.5817,1.5540,-0.0764,-0.0323,1.2390,...,-0.7250,-0.6297,0.6103,0.0223,-1.3240,-0.3174,-0.6417,-0.2187,-1.4080,0.6931
3,id_0015fd391,trt_cp,48,D1,-0.5138,-0.2491,-0.2656,0.5288,4.0620,-0.8095,...,-2.0990,-0.6441,-5.6300,-1.3780,-0.8632,-1.2880,-1.6210,-0.8784,-0.3876,-0.8154
4,id_001626bd3,trt_cp,72,D2,-0.3254,-0.4009,0.9700,0.6919,1.4180,-0.8244,...,0.0042,0.0048,0.6670,1.0690,0.5523,-0.3031,0.1094,0.2885,-0.3786,0.7125


In [11]:
df_stats(train_df)

,カラム名,カラムごとのユニーク値数,最も出現頻度の高い値,最も出現頻度の高い値の出現回数,欠損損値の割合,最も多いカテゴリの割合,Type
0,sig_id,23814,id_310db0a74,1,0.0,0.004199,object
1,cp_type,2,trt_cp,21948,0.0,92.164273,object
2,cp_time,3,48,8250,0.0,34.643487,int64
3,cp_dose,2,D1,12147,0.0,51.007811,object
4,g-0,14367,0,22,0.0,0.092383,float64
...,...,...,...,...,...,...,...
871,c-95,14693,-10,53,0.0,0.222558,float64
872,c-96,14493,-10,385,0.0,1.616696,float64
873,c-97,14757,-10,53,0.0,0.222558,float64
874,c-98,14812,-10,79,0.0,0.331738,float64


In [12]:
c_cols = [col for col in train_df.columns if col.startswith("c-")]

In [27]:
train_df.groupby('cp_dose').mean().loc[:, c_cols].describe()

,c-0,c-1,c-2,c-3,c-4,c-5,c-6,c-7,c-8,c-9,...,c-90,c-91,c-92,c-93,c-94,c-95,c-96,c-97,c-98,c-99
count,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,...,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000
mean,-0.354309,-0.454221,-0.479580,-0.376164,-0.492563,-0.445015,-0.573526,-0.265657,-0.450326,-0.457389,...,-0.468371,-0.460354,-0.511569,-0.498887,-0.505844,-0.352900,-0.462401,-0.377409,-0.469072,-0.300760
std,0.059443,0.079859,0.067493,0.105397,0.069422,0.071713,0.093255,0.044654,0.068873,0.089426,...,0.061220,0.074104,0.118348,0.088070,0.087639,0.057896,0.076054,0.058425,0.082803,0.052275
min,-0.396341,-0.510689,-0.527305,-0.450690,-0.541652,-0.495723,-0.639467,-0.297232,-0.499026,-0.520623,...,-0.511660,-0.512754,-0.595254,-0.561162,-0.567814,-0.393839,-0.516180,-0.418721,-0.527623,-0.337724
25%,-0.375325,-0.482455,-0.503443,-0.413427,-0.517107,-0.470369,-0.606496,-0.281444,-0.474676,-0.489006,...,-0.490016,-0.486554,-0.553411,-0.530025,-0.536829,-0.373370,-0.489290,-0.398065,-0.498347,-0.319242
50%,-0.354309,-0.454221,-0.479580,-0.376164,-0.492563,-0.445015,-0.573526,-0.265657,-0.450326,-0.457389,...,-0.468371,-0.460354,-0.511569,-0.498887,-0.505844,-0.352900,-0.462401,-0.377409,-0.469072,-0.300760
75%,-0.333293,-0.425986,-0.455718,-0.338900,-0.468018,-0.419660,-0.540555,-0.249869,-0.425975,-0.425773,...,-0.446727,-0.434155,-0.469727,-0.467750,-0.474859,-0.332431,-0.435512,-0.356752,-0.439797,-0.282278
max,-0.312277,-0.397752,-0.431856,-0.301637,-0.443474,-0.394306,-0.507585,-0.234082,-0.401625,-0.394156,...,-0.425082,-0.407955,-0.427885,-0.436612,-0.443874,-0.311962,-0.408623,-0.336096,-0.410521,-0.263796


In [14]:
for c in c_cols:
    train_df.loc[:, c + "-time"] = train_df.loc[:, c] / train_df.loc[:, "cp_time"]

In [15]:
c_ave_cols = [col for col in train_df.columns if col.endswith("-time")]

In [17]:
train_c_ave_df = train_df.groupby('cp_time').mean().loc[:, c_ave_cols]

In [18]:
train_c_ave_df.describe()

,c-0-time,c-1-time,c-2-time,c-3-time,c-4-time,c-5-time,c-6-time,c-7-time,c-8-time,c-9-time,...,c-90-time,c-91-time,c-92-time,c-93-time,c-94-time,c-95-time,c-96-time,c-97-time,c-98-time,c-99-time
count,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,...,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000
mean,-0.007277,-0.010224,-0.010571,-0.007985,-0.010443,-0.009402,-0.013005,-0.005790,-0.009677,-0.010404,...,-0.010697,-0.010505,-0.011251,-0.011969,-0.011292,-0.006824,-0.010248,-0.008268,-0.010033,-0.006449
std,0.000616,0.002280,0.001783,0.000528,0.000509,0.000465,0.003248,0.000895,0.000897,0.002606,...,0.002970,0.002798,0.001784,0.004856,0.002269,0.001856,0.001911,0.001194,0.000780,0.000525
min,-0.007882,-0.012725,-0.012594,-0.008392,-0.011008,-0.009735,-0.016674,-0.006815,-0.010413,-0.013198,...,-0.014095,-0.013646,-0.013280,-0.017429,-0.013816,-0.007927,-0.012433,-0.009530,-0.010651,-0.007029
25%,-0.007592,-0.011206,-0.011243,-0.008284,-0.010655,-0.009667,-0.014260,-0.006106,-0.010177,-0.011587,...,-0.011745,-0.011618,-0.011912,-0.013887,-0.012228,-0.007895,-0.010924,-0.008825,-0.010471,-0.006670
50%,-0.007302,-0.009688,-0.009891,-0.008175,-0.010302,-0.009600,-0.011845,-0.005397,-0.009940,-0.009976,...,-0.009394,-0.009590,-0.010544,-0.010344,-0.010640,-0.007863,-0.009416,-0.008119,-0.010290,-0.006310
75%,-0.006975,-0.008974,-0.009560,-0.007782,-0.010160,-0.009235,-0.011171,-0.005278,-0.009309,-0.009008,...,-0.008998,-0.008935,-0.010236,-0.009239,-0.010031,-0.006272,-0.009155,-0.007638,-0.009723,-0.006159
max,-0.006649,-0.008260,-0.009228,-0.007389,-0.010018,-0.008870,-0.010496,-0.005158,-0.008678,-0.008040,...,-0.008601,-0.008279,-0.009928,-0.008134,-0.009421,-0.004681,-0.008894,-0.007156,-0.009156,-0.006007


In [19]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()


In [22]:
import pandas as pd

In [23]:
train_c_ave_df = pd.DataFrame(scaler.fit_transform(train_c_ave_df))

In [24]:
train_c_ave_df

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,1.248014,-1.343116,-1.389558,-0.439125,-1.358082,-0.522318,-1.383395,-1.401641,-1.004805,-1.312971,...,-1.401528,-1.374858,-1.393018,-1.377117,-1.362217,1.414009,-1.400971,-1.293982,-0.971381,-1.353926
1,-1.200068,0.288113,0.467081,-0.944644,0.337413,-0.876992,0.437409,0.863766,-0.359442,0.201424,...,0.537087,0.400506,0.485262,0.409879,0.352058,-0.727859,0.533270,0.152816,-0.404430,0.323180
2,-0.047947,1.055003,0.922477,1.383769,1.020669,1.399310,0.945986,0.537875,1.364247,1.111547,...,0.864441,0.974351,0.907756,0.967237,1.010159,-0.686150,0.867700,1.141165,1.375811,1.030746


In [25]:
train_c_ave_df.describe()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
count,3.000000e+00,3.000000,3.000000,3.000000,3.000000e+00,3.000000e+00,3.000000,3.000000e+00,3.000000,3.000000e+00,...,3.000000e+00,3.000000,3.000000,3.000000,3.000000e+00,3.000000e+00,3.000000e+00,3.000000,3.000000,3.000000
mean,-7.170190e-17,0.000000,0.000000,0.000000,-7.401487e-17,-7.401487e-17,0.000000,-3.700743e-17,0.000000,7.401487e-17,...,-3.700743e-17,0.000000,0.000000,0.000000,7.401487e-17,-3.700743e-17,3.700743e-17,0.000000,0.000000,0.000000
std,1.224745e+00,1.224745,1.224745,1.224745,1.224745e+00,1.224745e+00,1.224745,1.224745e+00,1.224745,1.224745e+00,...,1.224745e+00,1.224745,1.224745,1.224745,1.224745e+00,1.224745e+00,1.224745e+00,1.224745,1.224745,1.224745
min,-1.200068e+00,-1.343116,-1.389558,-0.944644,-1.358082e+00,-8.769920e-01,-1.383395,-1.401641e+00,-1.004805,-1.312971e+00,...,-1.401528e+00,-1.374858,-1.393018,-1.377117,-1.362217e+00,-7.278587e-01,-1.400971e+00,-1.293982,-0.971381,-1.353926
25%,-6.240070e-01,-0.527501,-0.461238,-0.691884,-5.103345e-01,-6.996551e-01,-0.472993,-4.318830e-01,-0.682123,-5.557735e-01,...,-4.322203e-01,-0.487176,-0.453878,-0.483619,-5.050796e-01,-7.070043e-01,-4.338502e-01,-0.570583,-0.687905,-0.515373
50%,-4.794652e-02,0.288113,0.467081,-0.439125,3.374133e-01,-5.223181e-01,0.437409,5.378754e-01,-0.359442,2.014236e-01,...,5.370872e-01,0.400506,0.485262,0.409879,3.520575e-01,-6.861499e-01,5.332703e-01,0.152816,-0.404430,0.323180
75%,6.000338e-01,0.671558,0.694779,0.472322,6.790411e-01,4.384960e-01,0.691698,7.008207e-01,0.502402,6.564853e-01,...,7.007639e-01,0.687429,0.696509,0.688558,6.811083e-01,3.639293e-01,7.004853e-01,0.646991,0.485690,0.676963
max,1.248014e+00,1.055003,0.922477,1.383769,1.020669e+00,1.399310e+00,0.945986,8.637660e-01,1.364247,1.111547e+00,...,8.644405e-01,0.974351,0.907756,0.967237,1.010159e+00,1.414009e+00,8.677004e-01,1.141165,1.375811,1.030746


In [13]:
test_df.head()

,sig_id,cp_type,cp_time,cp_dose,g-0,g-1,g-2,g-3,g-4,g-5,...,c-90,c-91,c-92,c-93,c-94,c-95,c-96,c-97,c-98,c-99
0,id_0004d9e33,trt_cp,24,D1,-0.5458,0.1306,-0.5135,0.4408,1.5500,-0.1644,...,0.0981,0.7978,-0.1430,-0.2067,-0.2303,-0.1193,0.0210,-0.0502,0.1510,-0.7750
1,id_001897cda,trt_cp,72,D1,-0.1829,0.2320,1.2080,-0.4522,-0.3652,-0.3319,...,-0.1190,-0.1852,-1.0310,-1.3670,-0.3690,-0.5382,0.0359,-0.4764,-1.3810,-0.7300
2,id_002429b5b,ctl_vehicle,24,D1,0.1852,-0.1404,-0.3911,0.1310,-1.4380,0.2455,...,-0.2261,0.3370,-1.3840,0.8604,-1.9530,-1.0140,0.8662,1.0160,0.4924,-0.1942
3,id_00276f245,trt_cp,24,D2,0.4828,0.1955,0.3825,0.4244,-0.5855,-1.2020,...,0.1260,0.1570,-0.1784,-1.1200,-0.4325,-0.9005,0.8131,-0.1305,0.5645,-0.5809
4,id_0027f1083,trt_cp,48,D1,-0.3979,-1.2680,1.9130,0.2057,-0.5864,-0.0166,...,0.4965,0.7578,-0.1580,1.0510,0.5742,1.0900,-0.2962,-0.5313,0.9931,1.8380


In [14]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3982 entries, 0 to 3981
Columns: 876 entries, sig_id to c-99
dtypes: float64(872), int64(1), object(3)
memory usage: 26.6+ MB


In [15]:
test_df[['c-31', 'c-32', 'c-78']].describe()

,c-31,c-32,c-78
count,3982.000000,3982.000000,3982.000000
mean,-0.404966,-0.326530,-0.398559
std,1.981752,1.816726,1.924608
min,-10.000000,-10.000000,-10.000000
25%,-0.512525,-0.531950,-0.554750
50%,0.015550,0.016800,-0.001100
75%,0.466400,0.495875,0.467600
max,4.541000,4.169000,5.597000


In [16]:
target_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23814 entries, 0 to 23813
Columns: 207 entries, sig_id to wnt_inhibitor
dtypes: int64(206), object(1)
memory usage: 37.6+ MB


In [17]:
target_df.head()

,sig_id,5-alpha_reductase_inhibitor,11-beta-hsd1_inhibitor,acat_inhibitor,acetylcholine_receptor_agonist,acetylcholine_receptor_antagonist,acetylcholinesterase_inhibitor,adenosine_receptor_agonist,adenosine_receptor_antagonist,adenylyl_cyclase_activator,...,tropomyosin_receptor_kinase_inhibitor,trpv_agonist,trpv_antagonist,tubulin_inhibitor,tyrosine_kinase_inhibitor,ubiquitin_specific_protease_inhibitor,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor
0,id_000644bb2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,id_000779bfc,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,id_000a6266a,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,id_0015fd391,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,id_001626bd3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
non_target_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23814 entries, 0 to 23813
Columns: 403 entries, sig_id to xiap_inhibitor
dtypes: int64(402), object(1)
memory usage: 73.2+ MB


In [19]:
non_target_df

,sig_id,abc_transporter_expression_enhancer,abl_inhibitor,ace_inhibitor,acetylcholine_release_enhancer,adenosine_deaminase_inhibitor,adenosine_kinase_inhibitor,adenylyl_cyclase_inhibitor,age_inhibitor,alcohol_dehydrogenase_inhibitor,...,ve-cadherin_antagonist,vesicular_monoamine_transporter_inhibitor,vitamin_k_antagonist,voltage-gated_calcium_channel_ligand,voltage-gated_potassium_channel_activator,voltage-gated_sodium_channel_blocker,wdr5_mll_interaction_inhibitor,wnt_agonist,xanthine_oxidase_inhibitor,xiap_inhibitor
0,id_000644bb2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,id_000779bfc,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,id_000a6266a,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,id_0015fd391,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,id_001626bd3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23809,id_fffb1ceed,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
23810,id_fffb70c0c,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
23811,id_fffc1c3f4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
23812,id_fffcb9e7c,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
non_target_df.describe()

,abc_transporter_expression_enhancer,abl_inhibitor,ace_inhibitor,acetylcholine_release_enhancer,adenosine_deaminase_inhibitor,adenosine_kinase_inhibitor,adenylyl_cyclase_inhibitor,age_inhibitor,alcohol_dehydrogenase_inhibitor,aldehyde_dehydrogenase_activator,...,ve-cadherin_antagonist,vesicular_monoamine_transporter_inhibitor,vitamin_k_antagonist,voltage-gated_calcium_channel_ligand,voltage-gated_potassium_channel_activator,voltage-gated_sodium_channel_blocker,wdr5_mll_interaction_inhibitor,wnt_agonist,xanthine_oxidase_inhibitor,xiap_inhibitor
count,23814.000000,23814.000000,23814.000000,23814.000000,23814.0,23814.000000,23814.000000,23814.000000,23814.000000,23814.000000,...,23814.000000,23814.000000,23814.000000,23814.0,23814.000000,23814.000000,23814.000000,23814.0,23814.000000,23814.000000
mean,0.000252,0.001008,0.003821,0.000252,0.0,0.000084,0.000252,0.000504,0.000504,0.000252,...,0.000252,0.000504,0.000756,0.0,0.001050,0.000756,0.000252,0.0,0.001008,0.001008
std,0.015871,0.031731,0.061700,0.015871,0.0,0.009164,0.015871,0.022443,0.022443,0.015871,...,0.015871,0.022443,0.027483,0.0,0.032384,0.027483,0.015871,0.0,0.031731,0.031731
min,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,0.0,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,0.0,1.000000,1.000000,1.000000,0.0,1.000000,1.000000


In [22]:
target_df.columns.tolist()

['sig_id',
 '5-alpha_reductase_inhibitor',
 '11-beta-hsd1_inhibitor',
 'acat_inhibitor',
 'acetylcholine_receptor_agonist',
 'acetylcholine_receptor_antagonist',
 'acetylcholinesterase_inhibitor',
 'adenosine_receptor_agonist',
 'adenosine_receptor_antagonist',
 'adenylyl_cyclase_activator',
 'adrenergic_receptor_agonist',
 'adrenergic_receptor_antagonist',
 'akt_inhibitor',
 'aldehyde_dehydrogenase_inhibitor',
 'alk_inhibitor',
 'ampk_activator',
 'analgesic',
 'androgen_receptor_agonist',
 'androgen_receptor_antagonist',
 'anesthetic_-_local',
 'angiogenesis_inhibitor',
 'angiotensin_receptor_antagonist',
 'anti-inflammatory',
 'antiarrhythmic',
 'antibiotic',
 'anticonvulsant',
 'antifungal',
 'antihistamine',
 'antimalarial',
 'antioxidant',
 'antiprotozoal',
 'antiviral',
 'apoptosis_stimulant',
 'aromatase_inhibitor',
 'atm_kinase_inhibitor',
 'atp-sensitive_potassium_channel_antagonist',
 'atp_synthase_inhibitor',
 'atpase_inhibitor',
 'atr_kinase_inhibitor',
 'aurora_kinase_inh